In [ ]:
!pip install alpaca-trade-api
!pip install praw
!pip install twitter
!pip install oauth
!pip install twitterscraper
import alpaca_trade_api as alpaca
import praw
import math
import time
import os

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 81kB 4.2MB/s 
     |████████████████████████████████| 163kB 4.4MB/s 
     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for oauth: filename=oauth-1.0.1-cp37-none-any.whl size=8005 sha256=25e3143cbb3bcb7bcea6919188dc0fda6c2771a1843fca30ec804168b3d0704e
  Stored in directory: /root/.cache/pip/wheels/f7/a8/ca/272f26e0c3973e23fe6720ca1c98fb2f5630263ab11c90af62
Successfully built oauth
     |████████████████████████████████| 92kB 3.7MB/s 
  Created wheel for twitterscraper: filename=twitterscraper-1.6.1-cp37-none-any.whl size=11347 sha256=ae1c413472ea1ac386457c18fa530669c0de5209e906bf940e816879c5e7cd5e
  Stored in directory: /root/.cache/pip/wheels/1a/50/a0/808fd6df0ac14c168b73874056a1ca52e4d88c0c1ecedd4e48
Successfully built twitterscraper


In [ ]:
api = alpaca.REST('PKXAG8XUHSF8596B6SHC', 'Kq5hVh1bdelVq1J0pj3zFv7HlgoOknHEe6WH4IBM', 'https://paper-api.alpaca.markets')

In [ ]:
############################################################################
# update these here to change the variables of the amazing trading algorithm
############################################################################

# positive_points will contain positive phrases with point values to add
# legends = (phrase to contain, must also contain, positive point value)
positive_points = (("💎", "", 1),
                   ("🚀", "", 1),
                   ("🦍", "", 1),
                   ("✊", "💎", 1),
                   ("👐", "💎", 1),
                   (" buying ", "I'm", 4),
                   (" holding ", "I'm", 4),
                   ("fuck the rich", "", 4),
                   ("to the moon", "", 4),
                   ("to the sun", "", 8),
                   (" buy ", "", 2),
                   (" hold ", "", 1),
                   (" never sell ", "", 10),
                   (" winning ", "", 2),
                   (" orbit ", "", 4),
                   (" profit ", "", 2),
                   ("i like the stock", "", 10))

In [ ]:
os.environ['APCA_API_BASE_URL'] = "https://paper-api.alpaca.markets"
os.environ['APCA_API_KEY_ID'] = "PK0WGJK9RCF2R1FM5KSG"
os.environ['APCA_API_SECRET_KEY'] = "tgIptafd1LzeoZk0CKvZH9KIx1PYbi365w1OjG6y"

reddit = praw.Reddit(client_id="rPbQL7re4oJKjQ",
                     client_secret="QS32mkQp4ntJ8iAIPDJNV6NBBLtUXw",
                     password="tpBCy9F5gZFRbdn",
                     user_agent="WSB analyses 1.0 by /u/United-Try1630",
                     username="United-Try1630"
                     )

# holds the values for each cycle
stock_points = {}

share_amounts = {}


def add_points(stock_ticker, point_value):
    if stock_ticker in stock_points:
        stock_points[stock_ticker] += point_value
    else:
        stock_points[stock_ticker] = point_value


def process_comment(comment):
    asset_list = api.list_assets(status='active')

    for point_string_list in positive_points:
        if point_string_list[0] in comment.body and point_string_list[1] in comment.body:
            for asset in asset_list:
                symbol_string = f" {asset.symbol} "
                if symbol_string in comment.body:
                    comment_score = comment.score - 4
                    point_value = point_string_list[2] * comment_score * comment.body.count(point_string_list[0])
                    add_points(asset.symbol, point_value)
                    print(asset.symbol, stock_points[asset.symbol])


def update_wsb_valuations():
  """check rolling comments on wsb new for our new values"""
  hot_posts = reddit.subreddit('wallstreetbets').hot(limit=10)

  for submission in hot_posts:
    print("new post")
    sub_comments = submission.comments
    sub_comments.replace_more(0)
    
    for comment in sub_comments:
      if comment.score > 5:
        process_comment(comment)
        continue


def get_account_value(account_value):
    for position in api.list_positions():
        price = api.get_last_trade(position.symbol).price
        account_value += float(price) * float(position.qty)
    return account_value


while True:
    stock_points.clear()

    if not api.get_clock().is_open:
        print('Market is not open.')
        time.sleep(300)
        continue

    if api.get_account().trading_blocked:
        print('Account is currently restricted from trading.')
        time.sleep(600)
        continue

    print("Begin updating...")
    update_wsb_valuations()

    account = api.get_account()
    account_value = float(account.buying_power)
    account_value = get_account_value(account_value)

    # get our stock amounts that we should be holding
    totalPoints = sum(stock_points.values())

    for ticker in stock_points:
        percentage = (stock_points[ticker] / totalPoints)
        moneyAmount = math.floor(percentage * account_value * .9)
        ticker_price = api.get_last_trade(ticker).price
        shareAmount = math.floor(moneyAmount / float(ticker_price))

        print(totalPoints,
              ticker,
              stock_points[ticker],
              moneyAmount,
              ticker_price,
              shareAmount,
              percentage,
              account_value)

        if shareAmount > 0:
            share_amounts[ticker] = shareAmount

    # sell anything we have too much of
    for position in api.list_positions():
        if position.symbol not in share_amounts:
            try:
                api.submit_order(
                    symbol=position.symbol,
                    qty=int(position.qty),
                    side='sell',
                    type='market',
                    time_in_force='day'
                )
            except:
              print("failed sell")
              pass

            time.sleep(2)
        elif share_amounts[position.symbol] < int(position.qty):
            # sell the difference
            try:
                api.submit_order(
                    symbol=position.symbol,
                    qty=(int(position.qty) - int(share_amounts[position.symbol])),
                    side='sell',
                    type='market',
                    time_in_force='day'
                )
            except:
                pass
            time.sleep(2)
    # wait and hopefully the orders goes through
    time.sleep(60)
    # buy the ones that I do need
    for ticker in share_amounts.keys():
        # if I already have some of this, see if i need to buy more
        found = False
        for position in api.list_positions():
            if ticker == position.symbol:
                if share_amounts[ticker] > int(position.qty):
                    found = True
                    # buy more shares!
                    try:
                        api.submit_order(
                            symbol=ticker,
                            qty=(int(share_amounts[ticker]) - int(position.qty)),
                            side='buy',
                            type='market',
                            time_in_force='day'
                        )
                    except:
                        print("failed buy")
                        pass
                    time.sleep(2)
        if not found:
            # if we don't already own it, let's buy some
            # buy more shares!
            try:
                api.submit_order(
                    symbol=ticker,
                    qty=int(share_amounts[ticker]),
                    side='buy',
                    type='market',
                    time_in_force='day'
                )
            except:
                pass
            time.sleep(2)

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Begin updating...


It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



new post
PLTR 38
SPY 16
AAPL 50
ZOM 6
HUGE 6
SQQQ 6
TSM 6
QS 4


It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



...

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



new post


It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



new post


HTTPError: ignored

In [ ]:
# from twitter import Twitter
# import oauth 

# con_secret = 'jmDWoiDYayKx7yl7Lbp85DwOe'
# con_secret_key='hD4SYCZ0HFSzXwYjRo9z2DcFtq1FceqqjaFUsdGQdcP0td68P0'
# token='2826412838-HmLSHhfDfQ1tRl4bApKRJC12qOD6fJoNlbgJ47r'
# token_key='yYso925Psbiyv5bhnB73JhDTE99Fctrbq6K1rXm84E0tT'

# twitter = Twitter(auth=oauth.twitterApi(token, token_key, con_secret, con_secret_key))

# # twitterApi = Twitter.api(consumer_key='jmDWoiDYayKx7yl7Lbp85DwOe',
# #                   consumer_secret='hD4SYCZ0HFSzXwYjRo9z2DcFtq1FceqqjaFUsdGQdcP0td68P0',
# #                   access_token_key='2826412838-HmLSHhfDfQ1tRl4bApKRJC12qOD6fJoNlbgJ47r',
# #                   access_token_secret='yYso925Psbiyv5bhnB73JhDTE99Fctrbq6K1rXm84E0tT')

# # requests.get(
# #     'https://api.twitter.com/1.1/search/tweets.json?q=tesla',
# #     headers={
# #         'authorization': 'Bearer '+BEARER_TOKEN
# # })

In [ ]:
  from twitterscraper.query import query_user_info
import pandas as pd
from multiprocessing import Pool
import time
from IPython.display import display


global twitter_user_info
twitter_user_info=[]


def get_user_info(twitter_user):
    """
    An example of using the query_user_info method
    :param twitter_user: the twitter user to capture user data
    :return: twitter_user_data: returns a dictionary of twitter user data
    """
    user_info = query_user_info(user= twitter_user)
    twitter_user_data = {}
    twitter_user_data["user"] = user_info.user
    twitter_user_data["fullname"] = user_info.full_name
    twitter_user_data["location"] = user_info.location
    twitter_user_data["blog"] = user_info.blog
    twitter_user_data["date_joined"] = user_info.date_joined
    twitter_user_data["id"] = user_info.id
    twitter_user_data["num_tweets"] = user_info.tweets
    twitter_user_data["following"] = user_info.following
    twitter_user_data["followers"] = user_info.followers
    twitter_user_data["likes"] = user_info.likes
    twitter_user_data["lists"] = user_info.lists
    
    return twitter_user_data


def main():
    start = time.time()
    users = ['Carlos_F_Enguix', 'mmtung', 'dremio', 'MongoDB', 'JenWike', 'timberners_lee','ataspinar2', 'realDonaldTrump',
            'BarackObama', 'elonmusk', 'BillGates', 'BillClinton','katyperry','KimKardashian']

    pool = Pool(8)    
    for user in pool.map(get_user_info,users):
        twitter_user_info.append(user)

    cols=['id','fullname','date_joined','location','blog', 'num_tweets','following','followers','likes','lists']
    data_frame = pd.DataFrame(twitter_user_info, index=users, columns=cols)
    data_frame.index.name = "Users"
    data_frame.sort_values(by="followers", ascending=False, inplace=True, kind='quicksort', na_position='last')
    elapsed = time.time() - start
    print(f"Elapsed time: {elapsed}")
    display(data_frame)
    

if __name__ == '__main__':
    main()